**Importa las bibliotecas necesarias.**

In [15]:
import psycopg2
from psycopg2 import Error # Importar la clase Error
from psycopg2 import sql
import pandas as pd

**Función para ejecutar una consulta SQL**

In [16]:
# Función para ejecutar una consulta SQL parametrizada
def execute_query(conn, query, params=None):
    """
    Ejecuta una consulta SQL con manejo de errores y soporte para parámetros.
    """
    try:
        with conn.cursor() as cursor:
            if params:
                cursor.execute(query, params)  # Pasar parámetros correctamente
            else:
                cursor.execute(query)
            
            conn.commit()
            print(f"Query executed successfully: {query[:50]}...")
            
            if cursor.description:
                return cursor.fetchall(), [desc[0] for desc in cursor.description]  # Obtener nombres de columnas
            else:
                return [], []  # Retornar listas vacías si no hay resultados
    except psycopg2.Error as err:
        conn.rollback()
        print(f"Error executing query: {err}")
        return [], []


**Conexión a la base de datos de PostgreSQL data_powergen**

In [17]:
# Conexión a la base de datos
def create_database_connection():
    conn = None
    try:
        conn = psycopg2.connect(
            database="data_powergen", 
            user="datafundamental_owner", 
            password="uozci0wQGpA6", 
            host="ep-solitary-pond-a2p3evpo.eu-central-1.aws.neon.tech", 
            sslmode="require", 
            port=5432
        )
        print("Connected to 'data_powergen' successfully.")
    except psycopg2.Error as err:
        print(f"Error: {err}")
    return conn

**Crea la tabla resultados_join y la lista de consultas a ejecutar.**

In [18]:
sql_queries = {
    "create_resultados_join": """
        CREATE TABLE IF NOT EXISTS resultados_join AS
        SELECT 
            h.id_equipo,
            COUNT(h.id_orden) AS total_ordenes_mantenimiento,
            AVG(r.temperatura_c) AS promedio_temperatura,
            AVG(r.vibracion_mm_s) AS promedio_vibracion,
            AVG(r.presion_bar) AS promedio_presion
        FROM historicos_ordenes h
        LEFT JOIN registros_condiciones r
        ON h.id_equipo = r.id_equipo
        GROUP BY h.id_equipo;
    """,
    "total_ordenes_por_equipo": """
        SELECT id_equipo, COUNT(id_orden) AS total_ordenes
        FROM historicos_ordenes
        GROUP BY id_equipo;
    """,
    "duracion_promedio_vida_util": """
        SELECT AVG(vida_util_anios) AS duracion_promedio_vida_util
        FROM caracteristicas_equipos;
    """,
    "costo_promedio_correctivo": """
        SELECT AVG(costo_mantenimiento) AS costo_promedio_correctivo
        FROM historicos_ordenes
        WHERE tipo_mantenimiento = 'Correctivo';
    """,
    "equipo_mas_fallas": """
        SELECT id_equipo, COUNT(id_orden) AS total_correctivo
        FROM historicos_ordenes
        WHERE tipo_mantenimiento = 'Correctivo'
        GROUP BY id_equipo
        ORDER BY total_correctivo DESC
        LIMIT 1;
    """,
    "temperatura_resumen": """
        SELECT id_equipo, AVG(temperatura_c) AS promedio_temp, MAX(temperatura_c) AS max_temp
        FROM registros_condiciones
        GROUP BY id_equipo;
    """,
    "ordenes_correctivas_equipo": """
        SELECT id_orden, id_equipo, costo_mantenimiento, tiempo_reparacion_horas
        FROM historicos_ordenes
        WHERE tipo_mantenimiento = 'Correctivo' AND id_equipo = %s;
    """,
    "dias_vibracion_temperatura": """
        SELECT *
        FROM registros_condiciones
        WHERE vibracion_mm_s > 3.0 AND temperatura_c > 60;
    """,
    "tiempo_promedio_mantenimiento": """
        WITH diferencias AS (
            SELECT 
                id_equipo,
                EXTRACT(EPOCH FROM AGE(fecha, LAG(fecha) OVER (PARTITION BY id_equipo ORDER BY fecha))) / 3600 AS tiempo_entre_mantenimientos
            FROM historicos_ordenes
        )
        SELECT 
            id_equipo, 
            AVG(tiempo_entre_mantenimientos) AS tiempo_promedio_horas
        FROM diferencias
        GROUP BY id_equipo;
    """,
    "suma_horas_acumuladas": """
        SELECT id_equipo, SUM(horas_acumuladas) AS total_horas_acumuladas
        FROM registros_condiciones
        GROUP BY id_equipo;
    """
}



**Ejecutar las consultas y mostrar resultados ordenados**

In [23]:
# Ejecutar las consultas y mostrar resultados ordenados
if __name__ == "__main__":
    conn = create_database_connection()
    if conn:
        for query_name, query in sql_queries.items():
            print(f"Executing query: {query_name}")
            result, columns = execute_query(conn, query)
            if result and columns:
                df = pd.DataFrame(result, columns=columns)
                df_sorted = df.sort_values(by=df.columns[0])  # Ordenar por la primera columna
                print(f"Resultados de '{query_name}':")
                display(df_sorted)  # Mostrar tabla ordenada en el notebook
            else:
                print(f"No data returned for query: {query_name}")

        # Cerrar conexión
        conn.close()
        print("Conexión cerrada.")

Connected to 'data_powergen' successfully.
Executing query: create_resultados_join
Query executed successfully: 
        CREATE TABLE IF NOT EXISTS resultados_joi...
No data returned for query: create_resultados_join
Executing query: total_ordenes_por_equipo
Query executed successfully: 
        SELECT id_equipo, COUNT(id_orden) AS tota...
Resultados de 'total_ordenes_por_equipo':


,id_equipo,total_ordenes
4,E001,487
19,E002,485
5,E003,506
14,E004,539
0,E005,502
9,E006,473
13,E007,485
11,E008,531
17,E009,507
10,E010,537


Executing query: duracion_promedio_vida_util
Query executed successfully: 
        SELECT AVG(vida_util_anios) AS duracion_p...
Resultados de 'duracion_promedio_vida_util':


,duracion_promedio_vida_util
0,19.0000000000000000


Executing query: costo_promedio_correctivo
Query executed successfully: 
        SELECT AVG(costo_mantenimiento) AS costo_...
Resultados de 'costo_promedio_correctivo':


...

Executing query: tiempo_promedio_mantenimiento
Query executed successfully: 
        WITH diferencias AS (
            SELECT ...
Resultados de 'tiempo_promedio_mantenimiento':


,id_equipo,tiempo_promedio_horas
0,E001,487.3580246913580247
1,E002,492.4958677685950413
2,E003,470.4475247524752475
3,E004,442.7063197026022305
4,E005,474.5868263473053892
5,E006,505.2711864406779661
6,E007,488.5289256198347107
7,E008,445.9924528301886792
8,E009,467.7154150197628458
9,E010,443.7313432835820896


Executing query: suma_horas_acumuladas
Query executed successfully: 
        SELECT id_equipo, SUM(horas_acumuladas) A...
Resultados de 'suma_horas_acumuladas':


,id_equipo,total_horas_acumuladas
4,E001,1109279
17,E002,1018046
5,E003,1321047
14,E004,1290416
0,E005,1159735
9,E006,1144940
13,E007,1025130
11,E008,1297036
19,E009,1010513
10,E010,1097655


Conexión cerrada.


** Ejecutar consulta parametrizada para un equipo específico**

In [24]:
 # Ejecutar consulta parametrizada para un equipo específico
        equipo_id = 'E005'
        result, columns = execute_query(conn, sql_queries["total_ordenes_por_equipo"])
        if result:
            df = pd.DataFrame(result, columns=columns)
            df_sorted = df.sort_values(by=df.columns[1], ascending=False)  # Ordenar por total_ordenes desc
            print("Total de órdenes por equipo:")
            display(df_sorted)

        # Cerrar conexión
        conn.close()
        print("Conexión cerrada.")

IndentationError: unexpected indent (4186978774.py, line 2)

**Ejecuta el código (conectar y consultar)**

In [5]:
# Ejecución del script
if __name__ == "__main__":
    # Conectar a la base de datos
    db_conn = create_database_connection()

    # Ejecutar cada consulta
    for query_name, query in sql_queries.items():
        print(f"Executing query: {query_name}")
        result = execute_query(db_conn, query)

        equipo_id = 'E005'
        result = execute_query(db_conn, sql_queries["ordenes_correctivas_equipo"], (equipo_id,))
        print(result)



        if result:  # Si la consulta devuelve resultados, imprimirlos
            print(result)

    # Cerrar la conexión
    db_conn.close()


Connected to 'data_powergen' successfully.
Executing query: create_resultados_join
Query executed successfully
Query executed successfully
[('OT000013', 'E005', Decimal('639'), Decimal('18')), ('OT000119', 'E005', Decimal('3672'), Decimal('26')), ('OT000121', 'E005', Decimal('907'), Decimal('29')), ('OT000224', 'E005', Decimal('3001'), Decimal('43')), ('OT000225', 'E005', Decimal('1680'), Decimal('30')), ('OT000226', 'E005', Decimal('1126'), Decimal('36')), ('OT000260', 'E005', Decimal('2127'), Decimal('15')), ('OT000322', 'E005', Decimal('3998'), Decimal('15')), ('OT000411', 'E005', Decimal('3982'), Decimal('28')), ('OT000478', 'E005', Decimal('3399'), Decimal('22')), ('OT000479', 'E005', Decimal('4206'), Decimal('15')), ('OT000571', 'E005', Decimal('4235'), Decimal('28')), ('OT000593', 'E005', Decimal('967'), Decimal('13')), ('OT000598', 'E005', Decimal('2048'), Decimal('39')), ('OT000629', 'E005', Decimal('2294'), Decimal('24')), ('OT000639', 'E005', Decimal('2473'), Decimal('11')),